# GMSO Centric Workflow for N-mer Partitioning 
---
This exercise is to demonstrate a simple workflow utilizing the entire set of MoSDeF tools to write out a simulation file to use in GROMACS. The main feature of this workflow is to demonstrate methods to use GMSO ForceFields and Topologies as the central data structures. This rerouted workflow, through a GMSO topology, gives the users the increased flexibility and functionality supported through GMSO. Although the workflow process remains relatively similar, differences allow users to gain increased functionality demonstrated in this workflow, such as using **forcefield matching** to individual molecules in the system, and to write **custom potential forms**. </br>

#### Note this workflow uses features that are still under development on the GMSO side of things. 
As such, it is necessary to install dev version of specific branches to access this functionality.
1. Install mBuild 
```bash
git clone https://github.com/mosdef-hub/mbuild.git
pip install -e ./
2. Install GMSO
```bash
git clone https://github.com/mosdef-hub/gmso.git
git pull develop
git checkout develop
pip install ./
```
3. Install forcefield-utilities
```bash
conda install forcefield-utilities -c conda-forge
```
___


### Exercise Stages:
1. Custom mBuild Recipes
    * Partitioned Box recipe to generate a structure with variable box length </br>
2. Build Simulation Box
    * Input parameters are polymer, box size, position of polymer. </br>
3. Parameterizion
    * Use direct forcefields for each molecule type. </br>
4. Write GROMACS files
    * init.top and init.gro will be written in the simulation directory gmso_files/gmso_sim/
    * Write mdp files. </br>
5. Run simulation
    * Run through equilibration and production of simulation. </br>
6. Evaluate Results
    * Use MDTraj to probe trajectory
---

# 1. Import Libraries
---

In [ ]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt

import mbuild as mb
import gmso
import forcefield_utilities as ffutils


# 1. Custom mBuild Recipes
---

In [ ]:
from scipy.constants import N_A
def Packing_Number(compound, vol):
    """
    Identify the number of compounds to place into a box.
    
    Parameters
    ----------
    compound: mbuild.Compound
    vol: float, int,
        Volume to fill
    
    NOTES
    -----
    The compound must have the attribute `compound.dens` which is used to
    calculate the number of compounds to fit into the volume.
    """
    n_compounds = compound.dens * vol / compound.mass * N_A * 1e-21
    return int(n_compounds)

In [ ]:
import operator
import functools
import mbuild as mb
def Partitioned_Box(solute, solvent1, solvent2, boxl, frac_interface=0):
    """
    Solubilize the `solute` into `solvent1` and fill other half of the 
    box, made as a cubic box with sidelengths `boxl` with `solvent2`.
    
    Parameters
    ----------
    solute: mb.Compound
        Solute to solvate one of
    solvent1: mb.Compound
        Solvent to put into one half of the box
    solvent2: mb.Compound
        Solvent to put into second half of the box
    boxl: float, int
        Cubic box length
    """
    # Pack mbuild box with water, polymer, and hexane
    full_box = mb.Box([boxl, boxl, boxl])
    half_box = mb.Box([boxl/2, boxl, boxl])
    vol=functools.reduce(operator.mul, full_box.lengths, 1)/2
    solute.translate(np.array([boxl+boxl*frac_interface, boxl, boxl])/2)

    filled_box1 = mb.packing.solvate(
        solvent=solvent1, 
        solute=solute, 
        box=half_box,  
        n_solvent=Packing_Number(solvent1, vol),
        edge=0.01
    )
    filled_box1.name = "sol1"
    filled_box2 = mb.packing.fill_box(
        compound=solvent2,
        box=half_box,  
        n_compounds=Packing_Number(solvent2, vol),
        edge=0.01
    )
    filled_box2.translate([boxl/2,0,0])
    filled_box2.name = "sol2"
    partitioned_box = mb.Compound()
    partitioned_box.add(filled_box1)
    partitioned_box.add(filled_box2)
    partitioned_box.box = full_box
    return partitioned_box

# 2. Build Simulation Box
---

In [ ]:
# Build Polymer
monomer = mb.load("CCCO", smiles=True) #PVA monomer
monomer.name = "monomer"
polymer= mb.lib.recipes.Polymer()
polymer.add_monomer(monomer, indices=(5,10)) #set indices of hydrogens to remove
polymer.build(n=2) #can change this to be any chain length
polymer.name = "polymer"
#polymer.energy_minimize() visualize with and without energy minimiization
polymer.visualize()

In [ ]:
# Build Solvent 1
water = mb.load("O", smiles=True)
water.name = 'water'
water.dens = 0.99 #set density property to use during packing

# Build Solvent 2
cyclopentane = mb.load("C1CCCC1", smiles=True)
cyclopentane.name = "cyclopentane"
cyclopentane.dens = 0.63

# Build Partitioned Box
boxl = 5 # Use a cubic boxlength of 5 nm
partitioned_box = Partitioned_Box(polymer, water, cyclopentane, boxl)
partitioned_box.visualize()

# 3. Parameterization
---

### Load forcefields

In [ ]:
import forcefield_utilities as ffutils

ffloader = ffutils.FoyerFFs() # ffloader is now an object where we can load in a forcefield for repeated uses. 
# In order to use a gmsoFF, we convert this Foyer forcefield to a GMSO forcefield.
polymer_ff = ffloader.load("gmso_files/alcohols.xml").to_gmso_ff()
pentane_ff = ffloader.load("gmso_files/alkanes.xml").to_gmso_ff()
water_ff = ffloader.load("gmso_files/tip3p.xml").to_gmso_ff() # 3 different foyer xmls located locally

### Convert Topology to GMSO

In [ ]:
from gmso.external import from_mbuild

topology_gmso = from_mbuild(partitioned_box) # Create GMSO topology
topology_gmso.identify_connections() # Identify angles and dihedrals (this is not optimized)
topology_gmso

### Apply forcefields using isomorphs

In [ ]:
from gmso.parameterization import apply
import warnings
warnings.simplefilter("ignore", UserWarning)

"""The reference names here are from the molecule names that were put into the box, 
and can be found by looking at topology_gmso.subtops"""
ff_dicts = {
    "water": water_ff,
    "cyclopentane": pentane_ff,
    "polymer": polymer_ff
} 

apply(
    topology_gmso, ff_dicts, identify_connected_components=True, use_molecule_info=False
) # apply forcefield to relevant subtops

print(len(topology_gmso.atom_types))
assert topology_gmso.is_fully_typed
topology_gmso

# 4. Write GROMACS Files
---
We will use the ParmEd object to write out these files, as the potential formats used in these forcefields are supported. However, for more complex potentials and as the GMSO writers are fully tested, you will be able to write out directly from the GMSO topology.

In [ ]:
%mkdir gmso_files/gmso_sim

"""GMSO Writing Methods, will not work with the develop
topology_gmso.save("gmso_files/gmso_sim/init.top")
topology_gmso.save("gmso_files/gmso_sim/init.gro")
"""
from gmso.external.convert_parmed import to_parmed
system = to_parmed(topology_gmso)
system.save("gmso_files/gmso_sim/init.top", overwrite=True)
system.save("gmso_files/gmso_sim/init.gro", overwrite=True)

In [ ]:
em_mdp = """
integrator          = steep
nsteps              = 500000
emstep              = 0.002
emtol               = 10
dt                  = 0.002

nstxout             = 10000
nstvout             = 10000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4

gen_vel             = yes
gen-temp            = 372.0
gen-seed            = 4

tcoupl              = no

pcoupl              = no

pbc                 = xyz

DispCorr            = EnerPres

constraint-algorithm = LINCS
constraints         = all-bonds
"""

nvt_mdp = """
integrator          = md
nsteps              = 1000000
dt                  = 0.001

comm-mode           = Linear

nstxout             = 10000
nstvout             = 10000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10
pbc                 = xyz

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4

tcoupl              = nose-hoover
tc-grps             = System
tau_t               = 1
ref_t               = 372.0

pcoupl              = no

DispCorr            = EnerPres

constraint-algorithm = LINCS
constraints         = all-bonds
"""

npt_mdp = """
integrator          = md
nsteps              = 1000000
dt                  = 0.001

comm-mode           = Linear

nstxout             = 1000
nstvout             = 1000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10
pbc                 = xyz 

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4 

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4 

gen_vel             = no

tcoupl              = nose-hoover
tc-grps             = System
tau_t               = 1 
ref_t               = 372.0

pcoupl                   = parrinello-rahman
pcoupltype               = isotropic
nstpcouple               = -1
tau-p                    = 10.0
compressibility          = 4.5e-5
ref-p                    = 14.02

DispCorr            = EnerPres

constraint-algorithm = LINCS
constraints         = all-bonds
"""

with open("gmso_files/gmso_sim/em.mdp", "w") as f:
    f.write(em_mdp)

with open("gmso_files/gmso_sim/nvt.mdp", "w") as f:
    f.write(nvt_mdp)

with open("gmso_files/gmso_sim/npt.mdp", "w") as f:
    f.write(npt_mdp)


# 5. Run Simulations
----
Files can be found in `gmso_files/gmso_sim`. These files are provided in this repository, but can be removed and regenerated with the below code.

In [ ]:
import os
if os.path.isfile("gmso_files/gmso_sim/npt.cpt"):
    raise OSError("Simulations are already run. Please delete gmso_files/gmso_sim to rerun simulation.")
    
%cd gmso_files/gmso_sim
!gmx grompp -f em.mdp -o em.tpr -c init.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm em -s em.tpr -cpi em.cpt

!gmx grompp -f nvt.mdp -o nvt.tpr -c em.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm nvt -s nvt.tpr -cpi nvt.cpt

!gmx grompp -f npt.mdp -o npt.tpr -c nvt.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm npt -s npt.tpr -cpi npt.cpt
%cd ../..

# 6. Data Visualization
-----
Brief examples of writing methods to evaluate at the data.

In [ ]:
def initialize_plot(title=None, xlabel=None, ylabel=None):
    fig, ax = plt.subplots(1, 1)

    ax.spines["bottom"].set_linewidth(3)
    ax.spines["left"].set_linewidth(3)
    ax.spines["right"].set_linewidth(3)
    ax.spines["top"].set_linewidth(3)
    ax.yaxis.tick_left()
    ax.yaxis.set_label_position('left')
    if title:
        ax.title.set_text(title)
    if xlabel:
        ax.set_xlabel(xlabel)
    if ylabel:
        ax.set_ylabel(ylabel)
    return fig, ax

In [ ]:
import numpy as np
import pylab as plt 
import panedr
from panedr import edr_to_df
# Plot Energy over time

data = edr_to_df("gmso_files/gmso_sim/npt.edr")

plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = initialize_plot(
    title='Control plot', xlabel=r"Time (ps)", ylabel='Energy $(kcal/mol)$'
)

dt, density = list(), list()
for i, j in enumerate(data["Total Energy"]):
    dt.append(i)
    density.append(j)
    
ax.plot(dt, density, "-", color='lightgray', label='Density')
ax.legend(loc="best")

In [ ]:
# Plot RDF
import mdtraj 
traj = mdtraj.load("gmso_files/gmso_sim/npt.trr", top="gmso_files/gmso_sim/init.gro")

# water heavy atoms
s1 = traj.topology.select("(resname =~ 'monom.*') and name C")
s2 = traj.topology.select("(resname =~ 'water.*') and name O")
pairs = traj.topology.select_pairs(s1, s2)
bins_wat, rdf_wat = mdtraj.compute_rdf(traj, pairs, r_range=(.1, 5))

# cyclopentant heavy atoms
s1 = traj.topology.select("(resname =~ 'monom.*') and name C")
s2 = traj.topology.select("(resname =~ 'cyclo.*') and name C")
pairs = traj.topology.select_pairs(s1, s2)
bins_pent, rdf_pent = mdtraj.compute_rdf(traj, pairs, r_range=(.1, 5))

fig, ax = initialize_plot(
    title='RDF of Polymer in Solvents', 
    xlabel=r"Distance (nm)", ylabel="g(R)"
)
ax.plot(bins_wat, rdf_wat, "-", color='blue', label='water')
ax.plot(bins_pent, rdf_pent, "-", color='green', label='pentane')
ax.legend(loc="best")

In [ ]:
# Plot Total Distributions across plot
s1 = traj.topology.select("all")
xpositions = traj.xyz[:,s1,:]
averaged_frames = np.average(xpositions, axis=0)


fig, ax = initialize_plot(
    title='Position Distributions Across Box', 
    xlabel=r"Distance (nm)", ylabel="Counts"
)
ax.hist(averaged_frames, bins=25, label=["x", "y", "z"]) # solvent is distributed across x axis
ax.legend(loc="best") 

In [ ]:
# Plot Water Distributions across plot

s1 = traj.topology.select("(resname =~ 'water.*') and name O")
s2 = traj.topology.select("(resname =~ 'cyclo.*') and name C")
s3 = traj.topology.select("(resname =~ 'monom.*') and name C")

positions = {
    "water": traj.xyz[:,s1,0],
    "pentane":traj.xyz[:,s2,0],
    "polymer":traj.xyz[:,s3,0]
}

fig, ax = initialize_plot(
    title='Position Distributions Per Molecule', 
    xlabel=r"Distance (nm)", ylabel="Normalized Counts"
)

for name, pos in positions.items():
    data, bins = np.histogram(pos, bins=100)
    centers = bins[0:-1] + (bins[1]-bins[0])/2
    ax.plot(centers, data/max(data), label=name)
ax.legend(loc="best")
fig.show()

In [ ]:
def get_interface(positionsDict, label, interface_fraction, frame, n_bins):
    pos = positionsDict[label][frame]
    data, bins = np.histogram(pos, bins=n_bins)
    centers = bins[0:-1] + (bins[1]-bins[0])/2
    midvalue = (np.max(data) - np.min(data))*interface_fraction 
    remove_bounds = (int(n_bins*0.3), int(n_bins*0.7)) # keep interface in center 40% of box
    mid_idx = (np.abs(data[remove_bounds[0]:remove_bounds[1]] - midvalue)).argmin() + remove_bounds[0]
    #print(mid_idx-n_bins/2)
    assert np.isclose(n_bins/2, mid_idx, atol=n_bins*0.25) # check position of interface
    midpoint = centers[mid_idx]
    return midpoint

In [ ]:
def evaluate_partition_fraction(positionsDict, n_bins=50, interface_fraction=0.8):
    partition_counter = {"water":0, "pentane":0, "interface":0}
    keysList = list(positionsDict.keys())
    
    frames = len(positionsDict[keysList[0]])
    for frame in range(frames):
        water_interface = get_interface(positionsDict, "water", interface_fraction, frame, n_bins=n_bins)
        pentane_interface = get_interface(positionsDict, "pentane", interface_fraction, frame, n_bins=n_bins)
        polymer_position = positionsDict["polymer"].mean()
        
        # Evaluate if water_interface or pentane_interface is greater
        if water_interface > pentane_interface:
            if polymer_position < pentane_interface:
                partition_counter["pentane"] += 1
            elif polymer_position > water_interface:
                partition_counter["water"] += 1
            else:
                partition_counter["interface"] += 1
        else:
            if polymer_position > pentane_interface:
                partition_counter["pentane"] += 1
            elif polymer_position < water_interface:
                partition_counter["water"] += 1
            else:
                partition_counter["interface"] += 1
    
    return partition_counter

evaluate_partition_fraction(positions, interface_fraction=0.5, n_bins=50)

In [ ]:
water_counts = []
pentane_counts = []
fracs = np.arange(0.1,0.6,0.02)
for frac in fracs:
    interface_data = evaluate_partition_fraction(positions, n_bins=50, interface_fraction=frac)
    water_counts.append(interface_data["water"])
    pentane_counts.append(interface_data["pentane"])
fig, ax = initialize_plot(
    title='Position Distributions Per Molecule', 
    xlabel=r"Fraction of Bulk Solvent Considered the Interface Position", ylabel="Number of Associated Molecules"
)
ax.scatter(fracs, water_counts, label="water")
ax.scatter(fracs, pentane_counts, marker="*", label="pentane")
ax.legend(loc="best")
fig.show()